# Unit 3: Programming with RDDs

## Contents
```
3.1 Before we begin: Passing funtions to Spark
3.2 Transformations
3.3 Actions
3.4 Loading data from HDFS
3.5 Saving results back to HDFS
```

## Before we begin: Passing functions to Spark

Using lambda functions:

In [1]:
rdd1 = sc.parallelize(range(4))
rdd1.collect()

[0, 1, 2, 3]

In [2]:
rdd2 = rdd1.map(lambda x: 2*x)
rdd2.collect()

[0, 2, 4, 6]

Using normal functions:

In [3]:
def double(x):
    return 2*x

In [4]:
rdd3 = rdd1.map(double)
rdd3.collect()

[0, 2, 4, 6]

Sometimes it is tricky to understand the scope and life cycle of variables and methods when running in a cluster. The main part of the code executes in the driver, but when parallel operations are done the functions passed are executed in the executors and data is passed around using **closures**.

## Transformations

### map

In [5]:
rdd1 = sc.parallelize(range(4))
rdd1.collect()

[0, 1, 2, 3]

In [6]:
rdd2 = rdd1.map(lambda x: x + 5)
rdd2.collect()

[5, 6, 7, 8]

In [7]:
def plus_five(x):
    return x + 5

rdd1.map(plus_five).collect()

[5, 6, 7, 8]

### filter

In [8]:
rdd1 = sc.parallelize(['a1', 'a2', 'b1', 'b2'])
rdd1.collect()

['a1', 'a2', 'b1', 'b2']

In [9]:
rdd2 = rdd1.filter(lambda x: 'a' in x)
rdd2.collect()

['a1', 'a2']

### flatMap

In [10]:
rdd1 = sc.parallelize(['Space: the final frontier.',
                       'These are the voyages of the starship Enterprise.'])
rdd1.collect()

['Space: the final frontier.',
 'These are the voyages of the starship Enterprise.']

In [11]:
rdd2 = rdd1.map(lambda line: line.split())
rdd2.collect()

[['Space:', 'the', 'final', 'frontier.'],
 ['These', 'are', 'the', 'voyages', 'of', 'the', 'starship', 'Enterprise.']]

In [12]:
rdd3 = rdd1.flatMap(lambda line: line.split())
rdd3.collect()

['Space:',
 'the',
 'final',
 'frontier.',
 'These',
 'are',
 'the',
 'voyages',
 'of',
 'the',
 'starship',
 'Enterprise.']

### distinct

In [13]:
rdd1 = sc.parallelize([1, 1, 1, 2, 2])
rdd1.collect()

[1, 1, 1, 2, 2]

In [14]:
rdd2 = rdd1.distinct()
rdd2.collect()

[2, 1]

## Actions

In [15]:
rdd1 = sc.parallelize([1, 1, 1, 2, 2])

### reduce

In [16]:
rdd1.reduce(lambda a, b: a + b)

7

### count

In [17]:
rdd1.count()

5

### collect

In [18]:
rdd1.collect()

[1, 1, 1, 2, 2]

### first

In [19]:
rdd1.first()

1

### take

In [20]:
rdd1.take(2)

[1, 1]

### takeSample

In [21]:
rdd1.takeSample(withReplacement=False, num=10)

[2, 1, 1, 2, 1]

In [22]:
rdd1.takeSample(withReplacement=True, num=10)

[1, 2, 1, 1, 1, 1, 1, 1, 1, 2]

## Loading data from HDFS

### textFile

In [23]:
rdd = sc.textFile('datasets/meteogalicia.txt')

In [24]:

rdd.take(5)

[u'',
 u'',
 u'ESTACI\ufffdN AUTOM\ufffdTICA:Santiago-EOAS',
 u'CONCELLO:Santiago de Compostela',
 u'PROVINCIA:A Coru\ufffda']

Several files can also be loaded together at the same time but **be careful with the number of partitions generated**:

In [25]:
rdd1 = sc.textFile('datasets/slurmd/slurmd.log.*')

In [26]:
print rdd1.toDebugString()

(10) datasets/slurmd/slurmd.log.* MapPartitionsRDD[29] at textFile at NativeMethodAccessorImpl.java:0 []
 |   datasets/slurmd/slurmd.log.* HadoopRDD[28] at textFile at NativeMethodAccessorImpl.java:0 []


In [27]:
rdd1.takeSample(withReplacement=False, num=5)

[u'1488161034 2017 Feb 27 03:03:54 c6610 daemon info slurmd Launching batch job 467165 for UID 1053',
 u'1494997188 2017 May 17 06:59:48 c6603 user info slurmstepd task/cgroup: /slurm/uid_12329/job_706187/step_batch: alloc=16384MB mem.limit=16384MB memsw.limit=unlimited',
 u'1486762787 2017 Feb 10 22:39:47 c6604 daemon info slurmd _run_prolog: run job script took usec=39335',
 u'1492284836 2017 Apr 15 21:33:56 c6609 user info slurmstepd done with job',
 u'1489949176 2017 Mar 19 19:46:16 c6604 user info slurmstepd done with job']

### wholeTextFiles

wholeTextFiles lets you read a directory containing multiple small text files, and returns each of them as (filename, content) pairs. This is in contrast with textFile, which would return one record per line in each file.

In [28]:
rdd2 = sc.wholeTextFiles('datasets/slurmd/slurmd.log.*')

In [29]:
rdd2.toDebugString()

'(2) datasets/slurmd/slurmd.log.* MapPartitionsRDD[33] at wholeTextFiles at NativeMethodAccessorImpl.java:0 []\n |  WholeTextFileRDD[32] at wholeTextFiles at NativeMethodAccessorImpl.java:0 []'

In [30]:
rdd2.map(lambda (filename, content): (filename, content[:80])).collect()

[(u'hdfs://nameservice1/user/jlopez/datasets/slurmd/slurmd.log.c6601',
  u'1482336831 2016 Dec 21 17:13:51 c6601 daemon info slurmd launch task 387796.0 re'),
 (u'hdfs://nameservice1/user/jlopez/datasets/slurmd/slurmd.log.c6602',
  u'1482485639 2016 Dec 23 10:33:59 c6602 daemon info slurmd Slurmd shutdown complet'),
 (u'hdfs://nameservice1/user/jlopez/datasets/slurmd/slurmd.log.c6603',
  u'1482485628 2016 Dec 23 10:33:48 c6603 daemon info slurmd Slurmd shutdown complet'),
 (u'hdfs://nameservice1/user/jlopez/datasets/slurmd/slurmd.log.c6604',
  u'1482485636 2016 Dec 23 10:33:56 c6604 daemon info slurmd Slurmd shutdown complet'),
 (u'hdfs://nameservice1/user/jlopez/datasets/slurmd/slurmd.log.c6605',
  u'1482485640 2016 Dec 23 10:34:00 c6605 daemon info slurmd Slurmd shutdown complet'),
 (u'hdfs://nameservice1/user/jlopez/datasets/slurmd/slurmd.log.c6606',
  u'1482485652 2016 Dec 23 10:34:12 c6606 daemon info slurmd Slurmd shutdown complet'),
 (u'hdfs://nameservice1/user/jlopez/datasets/s

### binaryRecords

In [31]:
import struct
from collections import namedtuple

AcctRecord = namedtuple('AcctRecord',
                        'flag version tty exitcode uid gid pid ppid '
                        'btime etime utime stime mem io rw minflt majflt swaps '
                        'command')

def read_record(data):
    values = struct.unpack("2BH6If8H16s", data)
    return AcctRecord(*values)

In [32]:
raw_rdd = sc.binaryRecords('datasets/pacct-20160919', recordLength=64)
records = raw_rdd.map(read_record)
records.take(2)

[AcctRecord(flag=2, version=3, tty=0, exitcode=0, uid=0, gid=0, pid=24150, ppid=24144, btime=1474162981, etime=0.0, utime=0, stime=0, mem=3924, io=0, rw=0, minflt=482, majflt=0, swaps=0, command='accton\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'),
 AcctRecord(flag=0, version=3, tty=0, exitcode=0, uid=0, gid=0, pid=24151, ppid=24144, btime=1474162981, etime=0.0, utime=0, stime=0, mem=4300, io=0, rw=0, minflt=199, majflt=0, swaps=0, command='gzip\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00')]

## Saving results back to HDFS

In [33]:
rdd.saveAsTextFile('results_directory')

It will create a separate file for each partition of the RDD.

## Pipe RDDs to System Commands

A very interesting functionality of RDDs is that you can pipe the contents of the RDD to system commands, so you can easily parallelize the execution of common tasks in multiple nodes.

For each partition, all elements inside the partition are passed together (separated by newlines) as the stdin of the command, and each line of the stdout of the command will be transformed in one element of the output partition.

In [34]:
rdd = sc.parallelize(range(10), 4)

In [35]:
rdd.glom().collect()

[[0, 1], [2, 3], [4, 5], [6, 7, 8, 9]]

In [36]:
rdd.pipe('wc -l').collect()

[u'2', u'2', u'2', u'4']

In [37]:
rdd.pipe('cat').collect()

[u'0', u'1', u'2', u'3', u'4', u'5', u'6', u'7', u'8', u'9']

## Exercises
Now you can try to apply the above concepts to solve the following problems:
* Unit 3 Working with meteorological data 1
* Unit 3 Calculating Pi